In [1]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('data/wiggly_banana_preopt/deployed_opt.msh')
mat = elastic_rods.RodMaterial('+', 20000, 0.3, [5.0, 5.0, 0.2, 0.2], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

l.setPerSegmentRestLength(np.loadtxt('data/wiggly_banana_preopt/design_parameters.txt'))

jdo = l.dofOffsetForJoint(l.centralJoint())
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, np.deg2rad(80), fixedVars=fixedVars)
view = LinkageViewer(l, width=1024, labelOffset=-0.5)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [9]:
np.savetxt('normals_wiggly_preop.txt', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_wiggly_preop.msh')

In [3]:
import sys; sys.path.append('../..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

lopt = elastic_rods.RodLinkage('data/wiggly_banana_5/deployed_opt.msh')
lopt.setMaterial(mat)

lopt.setPerSegmentRestLength(np.loadtxt('data/wiggly_banana_5/design_parameters.txt'))

with suppress_stdout(): elastic_rods.compute_equilibrium(lopt, np.deg2rad(80), fixedVars=fixedVars)
viewopt = LinkageViewer(lopt, width=1024, labelOffset=-0.5)
viewopt.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
viewopt.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(DirectionalLight(color='white', intensity=0.6, positio…

In [4]:
jointIdxs, pre_opt_stress, parametric_coords = l.rodStresses()
jointIdxs, post_opt_stress, parametric_coords = lopt.rodStresses()

In [5]:
print("\n".join(["\t".join(map(str, jidxs)) for jidxs in jointIdxs]), file=open('wiggly_rod_joints.txt', 'w'))
from linkage_utils import writeRodSegments
writeRodSegments(l, 'wiggly_rod_segments.txt', zeroBasedIndexing=True)

In [8]:
np.savetxt('normals_postop.txt', np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_postop.msh')

In [6]:
from matplotlib import pyplot as plt

In [7]:
import os
try: os.mkdir('wiggly_stress')
except: pass
resolution = 1000
normalize = plt.Normalize(vmin=0, vmax=max(max(max(pre_opt_stress)), max(max(post_opt_stress))))
cmap = plt.cm.magma

def vis(name, stressValues):
    for ri in range(len(jointIdxs)):
        s = stressValues[ri]
        p = parametric_coords[ri][:]
        # add zero stress samples at the beginning and end of the rod
        s.insert(0, 0)
        s.append(0)
        p.insert(0, 0)
        p.append(np.ceil(p[-1]))
        interpolated_stresses = np.interp(np.linspace(0, p[-1], resolution), p, s)
        image = cmap(normalize(np.array(interpolated_stresses).reshape(1, resolution)))
        plt.imsave('wiggly_stress/{}_stress_{}.png'.format(name, ri), image)
vis('pre_opt', pre_opt_stress)
vis('post_opt', post_opt_stress)